# 0 bytes

In [3]:
import os
from tqdm import tqdm

PATH = 'data/voxceleb2/'

problems = []

for idx in tqdm(os.listdir(PATH)):
    currentidx_path = f'{PATH}{idx}/'
    for f in os.listdir(currentidx_path):
        currentfolder_path = f'{currentidx_path}{f}/'
        for w in os.listdir(currentfolder_path):
            current_file = f'{currentfolder_path}{w}'
            if os.stat(current_file).st_size == 0:
                problems.append(current_file)

100%|██████████| 6112/6112 [02:36<00:00, 38.96it/s]


In [6]:
print('Problems found:')
print(len(problems))
print('Folders:')
print(set([p.split('/')[2] for p in problems]))

{'id04762'}

# Slow loading

In [1]:
import os
from tqdm import tqdm

PATH = 'data/voxceleb2/'

problems = []

for idx in tqdm(os.listdir(PATH)):
    currentidx_path = f'{PATH}{idx}/'
    for f in os.listdir(currentidx_path):
        currentfolder_path = f'{currentidx_path}{f}/'
        for w in os.listdir(currentfolder_path):
            current_file = f'{currentfolder_path}{w}'
            if os.stat(current_file).st_size == 0:
                loadWAV(current_file)

100%|██████████| 6112/6112 [02:41<00:00, 37.85it/s]


In [2]:
from datasets.Sampler import Sampler
import torch
import importlib
from easydict import EasyDict

config = {
    "train_dataset": "VoxCeleb2",
    "train_list": "data/train_list.txt",
    "train_path": "data/voxceleb2/",
    "max_frames": 200,
    "max_epoch": 500,
    "batch_size": 400,
    "nDataLoaderThread": 5,
    "max_seg_per_spk": 500,
    "sampler": True,
    "nPerSpeaker": 2,
    "seed": 1337,

    "distributed": False,
}

config = EasyDict(config)

TrainDataset = importlib.import_module(
                'datasets.' + config.train_dataset).__getattribute__(config.train_dataset)
train_dataset = TrainDataset(**vars(config))
sampler = Sampler(train_dataset, **vars(config)) if config.sampler else None
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    num_workers=config.nDataLoaderThread,
    sampler=sampler,
    pin_memory=False,
    #worker_init_fn=worker_init_fn,
    drop_last=True,
)

times = []

In [4]:
import time

total_iterations=len(train_loader)
loop_time = time.time()
for x, y, f in train_loader:
    loop_time = time.time()-loop_time
    #times.append((loop_time, 0))
    print(f'Loop: {loop_time}s')
    loop_time = time.time()

Loop: 66.64511156082153s
Loop: 0.09994006156921387s
Loop: 0.046971797943115234s
Loop: 0.0s
Loop: 0.016985654830932617s
Loop: 12.579838752746582s


KeyboardInterrupt: 